# Troubleshooting the Stored Proc
I have had many cases where I've need to create some sort of dynamic stored proc - either because the client had choices in which columns they wanted to display or we needed to handle a "catch-all" query

**Scenario**
We have a catch all search query in our stored procedure, _<mark>Return_The_Catch_All_Search</mark>_, and we know there will be bugs in this. This notebook aims for us to do the following:
* Make sure we know what the different options and what are returned for each. 
* Use the @debug feature to troubleshoot issues.
* Set up reuseable code for fixing any bugs or adding new functionality for the stored procedure.


Run the query, displaying all the options and the defaults that are built into the parameters:

In [ ]:
EXEC Return_The_Catch_All_Search 
	@LastName	= NULL,
	@FirstName	= NULL,
	@AlterEgo	= NULL,
	@IncludePastEgos = 0, 
	@IncludeAvgCitizen = 1,
	@IncludeOrders	= 0,
	@ComicUniverse  = NULL

That's how the application will run it. There's a @debug parameter that's set to 0 by default. Here's what happens when you turn it on:

In [ ]:
EXEC Return_The_Catch_All_Search @Debug = 1

These scenarios should always be run to confirm if the procedure is returning data properly:

* @LastName returns just the last name.
* @IncludeAvgCitizen will include anyone listed as an Average Citizen.
* @IncludeOrders will add an additional field with the total of all of that person's order.
* @AlterEgo will return just the person who is that alter ego.
    * @IncludePastEgos will return everyone who had been that alter ego. The Alter_Ego_Name should display their current Alter_Ego_Name value.

Create a separate cell for each of these conditions to confirm.
*****************

* @LastName returns just the last name.

In [ ]:
EXEC Return_The_Catch_All_Search @LastName = 'Munroe'

* @IncludeAvgCitizen will include anyone listed as an Average Citizen.

In [ ]:
EXEC Return_The_Catch_All_Search @IncludeAvgCitizen = 1
EXEC Return_The_Catch_All_Search @includeAvgCitizen = 0

* @IncludeOrders will add an additional field with the total of all of that person's order.

In [ ]:
EXEC Return_The_Catch_All_Search @IncludeOrders = 1

* @AlterEgo will return just the person who is that alter ego.
    * @IncludePastEgos will return everyone who had been that alter ego. The Alter_Ego_Name should display their current Alter_Ego_Name value.

In [ ]:
EXEC Return_The_Catch_All_Search @AlterEgo = 'Wonder Woman'
EXEC Return_The_Catch_All_Search @AlterEgo = 'Wonder Woman', @IncludePastEgos = 1

That last result set failed. Debug to find out why:

In [ ]:
EXEC Return_The_Catch_All_Search @AlterEgo = 'Wonder Woman', @IncludePastEgos = 1, @debug = 1

Take the text that was returned from the output of the stored procedure and paste it below to test.

Use this one cell to test just this code to see what you were getting and what you should be without having to go back and forth between screens or constantly rerunning the original query. 

Now, which way do we want to do this?

In [ ]:
-- insert the code here. 

Now that we have the fix, we can work with the guts of the stored proc here:

In [ ]:
-- these are the stored proc parameters:
DECLARE 
	@LastName	varchar(50) = NULL,
	@FirstName	varchar(50) = NULL,
	@AlterEgo	varchar(50) = NULL,
	@IncludePastEgos bit = 0, 
	@IncludeAvgCitizen bit	= 1,
	@IncludeOrders	bit = 0,
	@ComicUniverse varchar(50) = NULL,
	@Debug bit = 1

-- inside of the stored proc is here:	
DECLARE @sql nvarchar(max)

SELECT @sql = '
SELECT p.First_Name, p.Last_Name, ae.Alter_Ego_Name' 
+ CASE WHEN @IncludeOrders = 1 
		THEN char(13) + char(10) + char(9) + ', SUM(Total_Due) as All_Orders_Total_Due'
		ELSE ''
	END
+ '
FROM Person as p
	JOIN ' 
+ CASE WHEN @IncludePastEgos = 1 AND @AlterEgo IS NOT NULL 
	THEN '(
		SELECT distinct Person_ID, Alter_Ego_ID 
		FROM Alter_Ego_Person FOR System_Time ALL
		) '
	ELSE 'Alter_Ego_Person '
	END
+ 'as aep ON p.Person_ID = aep.Person_ID
	JOIN Alter_Ego as ae ON ae.Alter_Ego_ID = aep.Alter_Ego_ID'
+ CASE WHEN @ComicUniverse IS NOT NULL 
	THEN char(13) + char(10) + char(9) + 'JOIN Comic_Universe as cu ON ae.Comic_Universe_ID = cu.Comic_Universe_ID'
	ELSE ''
END
+ CASE WHEN @IncludeOrders = 1
	THEN char(13) + char(10) + char(9) + 'LEFT JOIN Gadget_Order_Form as gof ON gof.Person_ID = p.Person_ID '
	ELSE ''
END
+ '
WHERE 1 = 1 ' -- adding this to make it easier to dynamically add WHERE conditions
+ CASE WHEN @LastName IS NOT NULL
	THEN char(13) + char(10) + 'AND p.Last_Name = ''' + TRIM(@LastName) + ''''
	ELSE ''
END
+ CASE WHEN @FirstName IS NOT NULL
	THEN char(13) + char(10) + 'AND p.First_Name = ''' + TRIM(@FirstName) + ''''
	ELSE ''
END
+ CASE WHEN @AlterEgo IS NOT NULL
	THEN char(13) + char(10) + 'AND ae.Alter_Ego_Name = ''' + TRIM(@AlterEgo) + ''''
	ELSE ''
END
+ CASE WHEN @ComicUniverse IS NOT NULL 
	THEN char(13) + char(10) + 'AND cu.Comic_Universe_Name = ''' + TRIM(@ComicUniverse) + ''''
	ELSE ''
END
+ CASE WHEN @IncludeAvgCitizen = 0
	THEN char(13) + char(10) + 'AND ae.Alter_Ego_Name <> ''Average Citizen'''
	ELSE ''
END 
+ '
GROUP BY p.First_Name, p.Last_Name, ae.Alter_Ego_Name
ORDER BY p.Last_Name'

IF @Debug = 1
	PRINT @sql

EXEC sp_executesql @sql

Now that the code is fixed and can be proved that it's fixed, transfer into the stored procedure script and run it. The file can be found at "..\Superhero Demo DB Setup\13 - PR - Return_The_Catch_All_Search.sql" (Adjust path for your setup.)

After the stored procedure has been fixed, rerun the test to confirm:

In [ ]:
EXEC Return_The_Catch_All_Search @AlterEgo = 'Wonder Woman'
EXEC Return_The_Catch_All_Search @AlterEgo = 'Wonder Woman', @IncludePastEgos = 1